## Step 4 : Using Python Wrapper googleplaces for Google Places API to get points of interest around stolen location

In [2]:
import numpy as np
import pandas as pd
#package for JSON interaction
import json
#package to interact with URL's
import urllib2

#package to using google api
import pygeocoder as geo

#package to get nearby places
import googleplaces as places

#time to sleep
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

Reading in data pulled for all bikes from BikeIndex.

In [2]:
df = pd.read_csv("bike_details_csv.csv")

Extracting just the latitude , longitude for every id

In [3]:
df_sub = df.loc[df.stolen == True, ['id','latitude','longitude']]

The process for pulling data from google places has a limit of 50K records per day so have to run this over multiple days. The below step puts in checks to pull in data for only bike id's not yet pulled.

In [3]:
#Getting id's already loaded
#df_loaded = pd.read_csv("places_details_csv.csv")
#df_loaded.columns = ['index','bike_id','google_place_id','place_lat','place_lng','place_name','place_types']

#df_loaded_id = df_loaded.bike_id.unique()
#len(df_loaded_id)

#df_sub = df_sub[~df_sub['id'].isin(df_loaded_id)]
#len(df_sub.id.unique())

In [7]:
df_sub = df_sub.reset_index()
df_sub=df_sub.drop('index',axis=1)

#sorting column to maintain integrity
df_sub.sort(columns='id',inplace=True)

#### Please key in your google places API key

In [4]:
#Google Places API key
YOUR_API_KEY = ''


The below function takes in bike record id, latitude , longitude and google places API key gets point of interest details for the selected radius of 1000 meters and writes it onto a CSV file.

In [10]:

def extract_places(bike_id,lat,lng,YOUR_API_KEY):
    
    google_places = places.GooglePlaces(YOUR_API_KEY)
    plc=google_places.nearby_search(lat_lng={'lat':lat,'lng':lng},radius=1000)
    
    places_df = pd.DataFrame()

    for place in plc.places:
        df_dict = {}
        df_dict['bike_id'] = bike_id
        df_dict['place_name'] = place.name
        df_dict['place_types'] = [place.types]
        df_dict['place_lat'] = place.geo_location['lat']
        df_dict['place_lng'] = place.geo_location['lng']
        df_dict['google_place_id'] = place.place_id

        df_temp = pd.DataFrame(df_dict)
    
        if places_df.shape[0] == 0:
            places_df = df_temp.copy()
        else:
            places_df = places_df.append(df_temp,ignore_index=True)
            
    col_order = ['bike_id','google_place_id','place_lat','place_lng','place_name','place_types']
        
    with open('places_details_csv.csv', 'a') as f:
        places_df.to_csv(f, header=False,encoding='utf8',columns=col_order)




The below loop runs through every record and calls the extract places function to pull points of interest information around every stolen location.

In [11]:
ctr = 0
for i, row in df_sub.iterrows():
    ctr = ctr+1
    lat = float(row['latitude'])
    lng = float(row['longitude'])
    bike_id = row['id']
    
    if (np.isnan(lat) == False and np.isnan(lat) == False):
        extract_places(bike_id,lat,lng,YOUR_API_KEY)
        
    if (ctr%100 == 0):
        print ctr




100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
